In [4]:
import datetime
import sqlite3 as sql 
import requests
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import json 
import networkx
from config_new import config
import time

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('omw-1.4')
if config['file_path']:
    os.chdir(config['file_path'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-d

In [3]:
bearertoken=config['bear_token']
headers = {'Authorization': 'Bearer ' +
       bearertoken, 'Content-Type': 'application/json'}
con = sql.connect(config['database'])

def init_users(con, headers):
    base_handels = {'NRSC':'NRSC',
        'DSCC':'DSCC',
        'DCCC':'DCCC',
        'NRCC':'NRCC',
        'RNC' :'GOP',
        'DNC':'DNC'}
    base_handels = ','.join(base_handels.values())
    BASE_URL_QUERY = f'https://api.twitter.com/2/users/by?usernames={base_handels}&user.fields=created_at&expansions=pinned_tweet_id&tweet.fields=author_id,created_at'
    auth_response_QUERY = requests.get(BASE_URL_QUERY,  headers=headers)
    auth_response_RESPONSE = json.loads(auth_response_QUERY.text)
    cur = con.cursor()
    cur.execute("""CREATE TABLE IF NOT EXISTS twitter_users (user_id INTEGER, user_name TEXT, parent_id INTEGER)
    """)
    con.commit()
    for i in auth_response_RESPONSE['data']:
        cur.execute(f"""INSERT INTO twitter_users (user_id, user_name, parent_id) VALUES ({i['id']},'{i['username']}',0)""")
        con.commit()

def get_follows(base_id, con, headers, next_token=None):
    cur = con.cursor()
    retry = True 
    while retry:   
        try:
            if next_token:
                BASE_URL_QUERY = f'https://api.twitter.com/2/users/11134252/following?user.fields=created_at&expansions=pinned_tweet_id&tweet.fields=created_at&max_results=1000&pagination_token={next_token}'
            else:
                BASE_URL_QUERY = f'https://api.twitter.com/2/users/{base_id}/following?user.fields=created_at&expansions=pinned_tweet_id&tweet.fields=created_at&max_results=1000'
            auth_response_QUERY = requests.get(BASE_URL_QUERY,  headers=headers)
            auth_response_RESPONSE = json.loads(auth_response_QUERY.text)
            if auth_response_RESPONSE['data']:
                for i in auth_response_RESPONSE['data']:
                    cur.execute(f"""INSERT INTO twitter_users (user_id, user_name, parent_id) VALUES ({i['id']},'{i['username']}', {base_id})""")
                    con.commit()
            if auth_response_QUERY.status_code == 429:
                time.sleep(900)
                get_follows(base_id, con, headers, next_token)
            elif 'next_token' in auth_response_RESPONSE['meta'].keys():
                get_follows(base_id, con, headers, auth_response_RESPONSE['meta']['next_token'])
            else:
                retry = False
        except:
            time.sleep(900)

# init_users(con, headers)
# df = pd.read_sql_query("""SELECT user_id from twitter_users""", con)
# user_ids = df['user_id'].tolist()
# for user_id in user_ids:
#     get_follows(user_id, con, headers)
    
# con.close()

In [ ]:
sql_latlonpop_query = '''SELECT STNAME, COUNAME, MAX(POPULATION) AS POPULATION, LATITUDE, LONGITUDE
                            FROM Count_Pop_Centroid
                            GROUP BY
                                STNAME'''

template_twitter_search = f'filter:verified near:{lat},{lon} within:10km'
headers = {'Authorization': 'Bearer ' +
    config['bear_token'], 'Content-Type': 'application/json'}
BASE_URL_QUERY = f'https://api.twitter.com/2/tweets/search/recent?query={template_twitter_search}&expansions=author_id&start_time={start_time}'
auth_response_QUERY = requests.get(BASE_URL_QUERY,  headers=headers)
res = auth_response_QUERY.json()